In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup


import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


# Helper Function

In [2]:
import re
def remove_numbers(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)
    return text


def remove_year(string):
    return re.sub(r'\s20\d{2}(\s|\-\d{2})?', ' ', string)

# Import Data 

In [3]:

dataset = pd.read_csv('/Users/bhumikornkongtaveelert/Documents/Spring 2024/CS 224N/final project/BERT-based classifier/BIL Launchpad Case studies - Sheet1.csv')

"""
Cleaning the data:
- only include columns we will use: 'Project Name', 'Project Description', 'Applicants', 'opportunitytitle', 'description'
- drop column with missing information
- currently, the grant opportunities include years, i will omit this from their title to group recurring grants together
"""
dataset = dataset[['Project Name', 'Project Description', 'Applicants', 'opportunitytitle', 'description']]
clean_dataset = dataset.dropna()
print(len(clean_dataset['opportunitytitle'].unique()))
# print(dataset['opportunitytitle'].unique())
clean_dataset['description'] = clean_dataset['description'].apply(remove_numbers)
clean_dataset['opportunitytitle'] = clean_dataset['opportunitytitle'].apply(remove_numbers)

print(len(clean_dataset['opportunitytitle'].unique()))
# print(dataset['opportunitytitle'].unique())

79
55


/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/2104849965.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataset['description'] = clean_dataset['description'].apply(remove_numbers)
/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/2104849965.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataset['opportunitytitle'] = clean_dataset['opportunitytitle'].apply(remove_numbers)


# Experiment 1: Classification of project <> grant name (multi-class text classification) (ignore description of program)

In [14]:
# Create new dataset of project + project name and grant title
simple_df = clean_dataset[['Project Name', 'Project Description', 'Applicants', 'opportunitytitle']]
simple_df['project_profile'] =  simple_df['Applicants'] + ': ' + simple_df['Project Description'] + ' ' + simple_df['Project Name'] 
simple_df = simple_df[['project_profile','opportunitytitle'] ]
possible_grants = simple_df['opportunitytitle'].unique()

#substitute label with number instead
label_dict = {}
for index, possible_label in enumerate(possible_grants):
    label_dict[possible_label] = index

simple_df['label'] = simple_df['opportunitytitle'].replace(label_dict)
simple_df.groupby(['opportunitytitle', 'label']).count()


/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/3795575931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_df['project_profile'] =  simple_df['Applicants'] + ': ' + simple_df['Project Description'] + ' ' + simple_df['Project Name']
/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/3795575931.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  simple_df['label'] = simple_df['opportunitytitle'].replace(label_dict)


,,project_profile
opportunitytitle,label,
Competitive Funding Opportunity: Pilot Program for Transit-Oriented Development (TOD) Planning,14,20
Tribal Transportation Program Safety Fund,8,175
*FY Notice of Funding Opportunity: Bipartisan Infrastructure Law (BIL) Airport Terminal Program (ATP) Grants,33,187
*FY Notice of Funding Opportunity: Bipartisan Infrastructure Law (BIL) FAA Contract Tower (FCT) Competitive Grant Program,36,53
*Grants for Buses and Bus Facilities Program,54,2
*INFRA Grants,12,20
Advanced Transportation Technologies and Innovative Mobility Deployment (ATTIMD) Program,17,8
"Bridge Investment Program - Planning, Bridge Projects, and Large Bridge Projects",19,37
FY Competitive Funding Opportunity: Public Transportation on Indian Reservations Program; Tribal Transit Program,15,39


In [90]:
simple_df['project_profile'][0]

'Fairbanks International Airport: This award funds the replacement of the passenger boarding bridge at Gate 3. Fairbanks International Airport'

# Experiment 2: Classification of project <> grant name (multi-class text classification) (with description of program)

In [ ]:
def remove_year(string):
    return re.sub(r'\s20\d{2}(\s|\-\d{2})?', ' ', string)

In [41]:
dataset = pd.read_csv('/Users/bhumikornkongtaveelert/Documents/Spring 2024/CS 224N/final project/BERT-based classifier/BIL Launchpad Case studies - Sheet1.csv')

"""
Cleaning the data:
- only include columns we will use: 'Project Name', 'Project Description', 'Applicants', 'opportunitytitle', 'description'
- drop column with missing information
- currently, the grant opportunities include years, i will omit this from their title to group recurring grants together
"""
dataset = dataset[['Project Name', 'Project Description', 'Applicants', 'opportunitytitle', 'description']]
clean_dataset = dataset.dropna()
print(len(clean_dataset['opportunitytitle'].unique()))
# print(dataset['opportunitytitle'].unique())
clean_dataset['description'] = clean_dataset['description'].apply(remove_numbers)
clean_dataset['opportunitytitle'] = clean_dataset['opportunitytitle'].apply(remove_numbers)

print(len(clean_dataset['opportunitytitle'].unique()))
# print(dataset['opportunitytitle'].unique())

simple_df = clean_dataset
simple_df['project_profile'] =  simple_df['Applicants'] + ': ' + simple_df['Project Description'] + ' ' + simple_df['Project Name'] 
simple_df['grant_profile'] =  simple_df['opportunitytitle'] + ': ' + simple_df['description'] 
simple_df = simple_df[['project_profile','grant_profile'] ]
possible_grants = simple_df['grant_profile'].unique()

#substitute label with number instead
label_dict = {}
for index, possible_label in enumerate(possible_grants):
    label_dict[possible_label] = index

simple_df['label'] = simple_df['grant_profile'].replace(label_dict)
simple_df['grant_profile'].unique()[0]
simple_df.groupby(['grant_profile']) #.to_csv('file.csv')

79
55


/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/2315656795.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataset['description'] = clean_dataset['description'].apply(remove_numbers)
/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/2315656795.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataset['opportunitytitle'] = clean_dataset['opportunitytitle'].apply(remove_numbers)
/var/folders/zn/y177ccm95vncshh465c0l2200000gn/T/ipykernel_43610/2315656795.py:2

In [43]:
simple_df['project_profile'].unique()

array(['Fairbanks International Airport: This award funds the replacement of the passenger boarding bridge at Gate 3. Fairbanks International Airport',
       'Ted Stevens Anchorage International Airport: This award funds the installation of 110 audio and visual monitors/stations in the terminal to better serve the passengers. Ted Stevens Anchorage International Airport',
       'Ted Stevens Anchorage International Airport: This award funds the installation of a new passenger boarding bridge at gate B4. Ted Stevens Anchorage International Airport',
       ...,
       "Southeastern Pennsylvania Transportation Authority: The Southeastern Pennsylvania Transportation Authority (SEPTA) will receive funding to buy up to 200 new rail cars to replace older rail cars that have been in service for nearly 25 years, operating along the Market Frankford Line, the most heavily used line in SEPTA's system. buy up to 200 new rail cars to replace older rail cars",
       "Colorado Department of Transpo

# Train validation split

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(simple_df.index.values, 
                                                  simple_df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=simple_df.label.values)

simple_df['data_type'] = ['not_set']*simple_df.shape[0]

simple_df.loc[X_train, 'data_type'] = 'train'
simple_df.loc[X_val, 'data_type'] = 'val'

simple_df.groupby(['grant_profile']).count()

KeyError: 'grant_profile'

# Tokenisation

In [7]:
"""
Constructs a BERT tokenizer. Based on WordPiece.
Instantiate a pre-trained BERT model configuration to encode our data.
To convert all the titles from text into encoded form, we use a function called batch_encode_plus , and we will proceed train and validation data separately.
The 1st parameter inside the above function is the title text.
add_special_tokens=True means the sequences will be encoded with the special tokens relative to their model.
When batching sequences together, we set return_attention_mask=True, so it will return the attention mask according to the specific tokenizer defined by the max_length attribute.
We also want to pad all the titles to certain maximum length.
We actually do not need to set max_length=256, but just to play it safe.
return_tensors='pt' to return PyTorch.
And then we need to split the data into input_ids, attention_masks and labels.
Finally, after we get encoded data set, we can create training data and validation data.
"""

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    simple_df[simple_df.data_type=='train'].project_profile.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    simple_df[simple_df.data_type=='val'].project_profile.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(simple_df[simple_df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(simple_df[simple_df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)


/Users/bhumikornkongtaveelert/miniconda3/envs/cs224n/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
attention_masks_train

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

# BERT

In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [10]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/Users/bhumikornkongtaveelert/miniconda3/envs/cs224n/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
from sklearn.metrics import f1_score
import numpy as np

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels, label_dict):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    class_accuracies = {}
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        accuracy = len(y_preds[y_preds == label]) / len(y_true)
        class_accuracies[label_dict_inverse[label]] = accuracy
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)} ({accuracy:.4f})\n')

    return class_accuracies

def recall_at_k(y_true, y_pred, k=5):
    num_samples = len(y_true)
    recall_count = 0
    
    for true_label, predictions in zip(y_true, y_pred):
        top_k_predictions = np.argsort(predictions)[-k:]  # Get indices of the top k predictions
        if true_label in top_k_predictions:
            recall_count += 1
    
    recall_at_k_score = recall_count / num_samples
    return recall_at_k_score

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total / len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    recall_at_5 = recall_at_k(true_vals, predictions, k=5)
    
    return loss_val_avg, predictions, true_vals, recall_at_5


In [13]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = 'cpu'
# !pip install ipywidgets
# ! pip install jupyter
# ! pip install ipywidgets widgetsnbextension pandas-profiling
# ! pip install jupyter_contrib_nbextensions
# ! jupyter contrib nbextension install --user
# !jupyter nbextension enable --py widgetsnbextension
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
     
    loss_val_avg, predictions, true_vals, recall_at_5 = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {loss_val_avg}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Recall@5: {recall_at_5}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1271 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.3909019886286687
Validation loss: 1.5648361675441265
F1 Score (Weighted): 0.5774398178794486
Recall@5: 0.8826151560178306


Epoch 2:   0%|          | 0/1271 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.287690200938174
Validation loss: 1.046124068022602
F1 Score (Weighted): 0.6552218122748082
Recall@5: 0.9479940564635958


Epoch 3:   0%|          | 0/1271 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Evaluate

# Predict

In [35]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
device = 'cpu'
model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_2.model', map_location=torch.device('cpu')))
# Example input texts
input_texts = ["This is the first example sentence.", "This is the second example sentence."]

# Tokenize the input texts
inputs = tokenizer(input_texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
inputs = {key: value.to(device) for key, value in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_classes = torch.argmax(probabilities, dim=1).cpu().numpy()
# Print the predicted classes
for text, predicted_class in zip(input_texts, predicted_classes):
    print(f'Text: "{text}" - Predicted class: {predicted_class}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: "This is the first example sentence." - Predicted class: 8
Text: "This is the second example sentence." - Predicted class: 8


In [45]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=device))
model.to(device)
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input texts
input_texts = ['Fairbanks International Airport: This award funds the replacement of the passenger boarding bridge at Gate 3. Fairbanks International Airport',
       'Ted Stevens Anchorage International Airport: This award funds the installation of 110 audio and visual monitors/stations in the terminal to better serve the passengers. Ted Stevens Anchorage International Airport',
       'Ted Stevens Anchorage International Airport: This award funds the installation of a new passenger boarding bridge at gate B4. Ted Stevens Anchorage International Airport',
       "Southeastern Pennsylvania Transportation Authority: The Southeastern Pennsylvania Transportation Authority (SEPTA) will receive funding to buy up to 200 new rail cars to replace older rail cars that have been in service for nearly 25 years, operating along the Market Frankford Line, the most heavily used line in SEPTA's system. buy up to 200 new rail cars to replace older rail cars",
       "Colorado Department of Transportation: The Colorado Department of Transportation, on behalf of Summit Stage, a rural transit agency serving Summit, Park and Lake counties, will receive funding to build a new charging and operation bus facility, which will include 100% electrification, fleet storage, and electric vehicle charging. The new facility will replace Summit Stage's current aging facility. Colorado Department of Transportation_CO_Buses and Bus Facilities Competitive Program",
       'Bi-State Development Agency: The Bi-State Development Agency of St. Louis will receive funding to transform bus stops into mobility hubs to improve safety and create a more welcoming experience for riders. The hubs will improve accessibility and include amenities such as benches, lighting and community information, at key transit transfer points. The BiState Development Agency of St Louis to transform bus stops into mobility hubs to improve safety and create a more welcoming experience for ride_MO']

# Tokenize and prepare input data
inputs = tokenizer(input_texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Extract top 5 predicted classes and their probabilities
top_k = 5
top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k, dim=1)
top_k_probabilities = top_k_probabilities.cpu().numpy()
top_k_indices = top_k_indices.cpu().numpy()

# Print the top 5 predicted classes and their probabilities for each input text
for text, indices, probs in zip(input_texts, top_k_indices, top_k_probabilities):
    print(f'Text: "{text}"')
    for idx, (pred_idx, prob) in enumerate(zip(indices, probs)):
        print(f'  Top {idx + 1}: {reversed_label_dict[pred_idx]} (Probability: {prob:.4f})')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: "Fairbanks International Airport: This award funds the replacement of the passenger boarding bridge at Gate 3. Fairbanks International Airport"
  Top 1: FY  Notice of Funding Opportunity: Bipartisan Infrastructure Law (BIL) Airport Terminal Program (ATP) Grants: The Department of Transportation (DOT), Federal Aviation Administration (FAA) announces the opportunity to apply for approximately $ billion in FY  discretionary funds for the Airport Terminal Program (ATP), made available under the Infrastructure Investment and Jobs Act of  (IIJA), Pub. L. -, herein referred to as the Bipartisan Infrastructure Law (BIL). The purpose of the ATP is to make annual grants available to eligible airports for airport terminal and airport-owned Airport Traffic Control Towers development projects that address the aging infrastructure of our nation&#;s airports. In addition, ATP grants will align with DOT&#;s Strategic Framework FY- at https://www.transportation.gov/administrations/office-policy/f

In [38]:
reversed_label_dict = {v: k for k, v in label_dict.items()}

# Print the reversed dictionary
print(reversed_label_dict)

{0: 'FY  Notice of Funding Opportunity: Bipartisan Infrastructure Law (BIL) Airport Terminal Program (ATP) Grants: The Department of Transportation (DOT), Federal Aviation Administration (FAA) announces the opportunity to apply for approximately $ billion in FY  discretionary funds for the Airport Terminal Program (ATP), made available under the Infrastructure Investment and Jobs Act of  (IIJA), Pub. L. -, herein referred to as the Bipartisan Infrastructure Law (BIL). The purpose of the ATP is to make annual grants available to eligible airports for airport terminal and airport-owned Airport Traffic Control Towers development projects that address the aging infrastructure of our nation&#;s airports. In addition, ATP grants will align with DOT&#;s Strategic Framework FY- at https://www.transportation.gov/administrations/office-policy/fy--strategic-framework. The FY  ATP will be implemented consistent with law and in alignment with the priorities in Executive Order , Implementation of th